# 2.2.2 Planzeichnung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [11]:
from utils.parser import Parser
from utils.openai import OpenAI
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()


In [12]:
%store -r msg211_f11_zecropbb_nz
ava.clearContext()
ava.addContext([
    # Output_1_PZE (best-case-scenario)
    {
        "type": "text",
        "text": f"Tabellenerklärung/Schema der Nutzungsschablone: {msg211_f11_zecropbb_nz}"
    },
    # Output_2_PZE (best-case-scenario)
    # {
    #     "type": "text",
    #     "text": 'Erklärung der Begriffe der Nutzungsschablone im JSON-Format: {"Art der Nutzung": {"Definition": "Gibt die Art der Nutzung des Grundstücks an, z.B. Gewerbegebiet.","Beispiel": "GE (Gewerbegebiet)"},"Gebäudehöhe": {"Definition": "Maximal zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe.","Beispiel": "GH=max.12,00"},"Grundflächenzahl": {"Definition": "Gibt das Verhältnis der bebauten Fläche zur Grundstücksfläche an.","Beispiel": "0,6"},"Geschossflächenzahl": {"Definition": "Gibt das Verhältnis der gesamten Geschossfläche zur Grundstücksfläche an.","Beispiel": "1,2"},"Bauweise": {"Definition": "Gibt die Art und Weise der Bebauung an, z.B. abweichende Bauweise.","Beispiel": "a (abweichende Bauweise)"}}'
    # },
])

In [6]:
# Planzeichnung: Nutzungsschablone – WITHOUT BOUNDING BOX
pz_crop_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-PZ-crop.png"

instruction = 'Gehe Schritt für Schritt vor: 1. Finde die Nutzungsschablonen in der vorliegenden Planzeichnung. 2. Gebe die Nutzungsschablone jeweils im JSON-Format aus.'

messages = await ava.extractTextFromImages(instruction, [pz_crop_path])
pprint(messages)

Schritt 1: Finde die Nutzungsschablonen in der vorliegenden Planzeichnung.

In der Planzeichnung sind mehrere Nutzungsschablonen zu erkennen. Diese sind durch Textfelder mit Angaben wie "GE", "GH=max.12,00", "0,6", "1,2" und weiteren Informationen gekennzeichnet.

Schritt 2: Gebe die Nutzungsschablone jeweils im JSON-Format aus.

Hier sind die Nutzungsschablonen im JSON-Format:

```json
[
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max.12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max.12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max.12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max.12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise

In [7]:
# Planzeichnung: Nutzungsschablone – WITH BOUNDING BOX
pz_bb_crop_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-PZ-bb-crop.png"

instruction = 'Gehe Schritt für Schritt vor: 1. Finde die Nutzungsschablonen in der vorliegenden Planzeichnung. 2. Gebe die Nutzungsschablone jeweils im JSON-Format aus.'

messages = await ava.extractTextFromImages(instruction, [pz_bb_crop_path])
pprint(messages)

Schritt 1: Finde die Nutzungsschablonen in der vorliegenden Planzeichnung.

In der Planzeichnung sind vier rot umrandete Nutzungsschablonen zu sehen. Diese enthalten Informationen zur Art der Nutzung, Gebäudehöhe, Grundflächenzahl, Geschossflächenzahl und Bauweise.

Schritt 2: Gebe die Nutzungsschablone jeweils im JSON-Format aus.

Hier sind die vier Nutzungsschablonen im JSON-Format:

```json
{
  "Nutzungsschablonen": [
    {
      "Art der Nutzung": "GEE",
      "Gebäudehöhe": "max. 12,00",
      "Grundflächenzahl": "0,6",
      "Geschossflächenzahl": "1,2",
      "Bauweise": "-"
    },
    {
      "Art der Nutzung": "GE",
      "Gebäudehöhe": "max. 12,00",
      "Grundflächenzahl": "0,6",
      "Geschossflächenzahl": "1,2",
      "Bauweise": "-"
    },
    {
      "Art der Nutzung": "GEE",
      "Gebäudehöhe": "max. 12,00",
      "Grundflächenzahl": "0,6",
      "Geschossflächenzahl": "1,2",
      "Bauweise": "-"
    },
    {
      "Art der Nutzung": "GE",
      "Gebäudehöhe": "max.

In [8]:
# Planzeichnung: Nutzungsschablone – VISUAL SEARCH + CROPPING
image_url_prompts = []
for index in range(4):
    base64_nz, img_size = parser.imageToBase64(f'../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-PZ-nz{index+1}-crop.png')
    image_url_prompts.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_nz}",
            "detail": "high"
        }
    })

message = ava.request([
    {
        "type": "text",
        "text": 'Hier sind die vorkommenden Nutzungsschablonen der Planzeichnung:'
    },
    *image_url_prompts,
    {
        "type": "text",
        "text": 'Gebe die Werte der jeweiligen Nutzungsschablonen strukturiert im JSON-Format aus: {<Parameter>:<Wert>}'
    }
])

print(message)

Hier sind die Werte der jeweiligen Nutzungsschablonen im JSON-Format:

```json
[
  {
    "Art der Nutzung": "GEE*",
    "Gebäudehöhe": "max. 12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "a"
  },
  {
    "Art der Nutzung": "GEE",
    "Gebäudehöhe": "max. 12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "a"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max. 12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "a"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max. 12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "a"
  }
]
```
